Building a First Model

In the last notebook, we loaded in the data from retrosheet and did some processing to get the team level statistics.
We saved that data to a file called 'df_bp1.csv'
In this notebook, we will load in that data, do some initital data exploration, and then build and evaluate our first predictive model

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import lightgbm as lgbm
import structureboost as stb
import ml_insights as mli

from structureboost import log_loss
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

In [3]:
df = pd.read_csv('df_bp1.csv')

/var/folders/w8/vvmn7s9x2yzffnkyk6j2j_xc0000gn/T/ipykernel_37817/2649927610.py:1: DtypeWarning: Columns (13,14,15,19,81,83,85,87,99,107,110,113,116,119,122,125,128,131,134,137,140,143,146,149,152,155,158) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('df_bp1.csv')


Data Checks/Exploration

In [ ]:
df.info(max_cols=1000)

In [ ]:
df.season.value_counts().sort_index()

In [9]:
hv_mean = df.home_victory.mean()
hv_mean

0.5408543584984002

In [ ]:
df.run_diff.value_counts()

In [ ]:
df.outs_total.value_counts()

In [ ]:
df.home_victory[df.outs_total==53].mean()

In [ ]:
df.loc[(df.outs_total==53) & (df.home_victory!=1),:]

In [ ]:
df.game_no_h.value_counts().sort_index()

In [ ]:
df[df.runs_v==df.runs_h]

BEGIN MODELING

In [4]:
df = df[df.run_diff!=0]
df_train = df[(df.season>1901) & (df.season<=2015)]
df_valid = df[(df.season>=2016) & (df.season<=2020)]
df_test = df[df.season>=2021]

In [5]:
features = ['OBP_162_h','OBP_162_v',
            'SLG_162_h','SLG_162_v', 
            # 'OBP_30_h','OBP_30_v',
            # 'SLG_30_h','SLG_30_v',
            # 'game_no_h',
           ]
target = 'home_victory'
X_train = df_train.loc[:,features]
X_valid = df_valid.loc[:,features]
X_test = df_test.loc[:,features]

y_train = df_train[target].to_numpy()
y_valid = df_valid[target].to_numpy()
y_test = df_test[target].to_numpy()


In [6]:
X_train.shape, X_valid.shape, X_test.shape, 

((188149, 4), (10615, 4), (4859, 4))

In [ ]:
plt.hist(X_train.OBP_162_h, np.linspace(.25,.4,151));

In [ ]:
plt.hist(X_train.SLG_162_v, np.linspace(.25,.5,201));

In [7]:
lgbm1 = lgbm.LGBMClassifier(n_estimators=1000, learning_rate=.02, max_depth=2)
lgbm1.fit(X_train, y_train, eval_set=(X_valid, y_valid), eval_metric='logloss', 
          callbacks=[lgbm.early_stopping(stopping_rounds=50), lgbm.log_evaluation(10)])

Training until validation scores don't improve for 50 rounds
[10]	valid_0's binary_logloss: 0.690165
[20]	valid_0's binary_logloss: 0.689465
[30]	valid_0's binary_logloss: 0.688776
[40]	valid_0's binary_logloss: 0.688155
[50]	valid_0's binary_logloss: 0.687758
[60]	valid_0's binary_logloss: 0.68738
[70]	valid_0's binary_logloss: 0.687123
[80]	valid_0's binary_logloss: 0.686874
[90]	valid_0's binary_logloss: 0.686538
[100]	valid_0's binary_logloss: 0.686386
[110]	valid_0's binary_logloss: 0.686215
[120]	valid_0's binary_logloss: 0.686012
[130]	valid_0's binary_logloss: 0.685846
[140]	valid_0's binary_logloss: 0.685752
[150]	valid_0's binary_logloss: 0.685638
[160]	valid_0's binary_logloss: 0.685541
[170]	valid_0's binary_logloss: 0.685487
[180]	valid_0's binary_logloss: 0.685428
[190]	valid_0's binary_logloss: 0.685351
[200]	valid_0's binary_logloss: 0.685303
[210]	valid_0's binary_logloss: 0.685251
[220]	valid_0's binary_logloss: 0.685213
[230]	valid_0's binary_logloss: 0.685169
[240]	

LGBMClassifier(learning_rate=0.02, max_depth=2, n_estimators=1000)

In [10]:
preds_lgbm = lgbm1.predict_proba(X_test)[:,1]
log_loss(y_test, preds_lgbm), log_loss(y_test, hv_mean*np.ones(len(y_test)))

(0.683490243306594, 0.690580943020142)

In [ ]:
rd = mli.get_range_dict(X_train)
plot_pts = X_test.sample(3, random_state=42)

In [ ]:
mli.ice_plot(lgbm1, plot_pts, X_test.columns, mli.get_range_dict(X_train), plots_per_row=2)

In [ ]:
plt.figure(figsize=(16,6))
mli.plot_reliability_diagram(y_test, preds_lgbm, show_histogram=True);

In [ ]:
fc = stb.get_basic_config(X_train, stb.default_config_dict())

stb1 = stb.StructureBoost(max_depth=3, learning_rate=.02, feature_configs = fc, num_trees=2000)
stb1.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stop_past_steps=5)

In [ ]:
preds_stb = stb1.predict(X_test)
log_loss(y_test, preds_stb), log_loss(y_test, hv_mean*np.ones(len(y_test)))

In [ ]:
plt.figure(figsize=(16,6))
mli.plot_reliability_diagram(y_test, preds_stb, show_histogram=True);

In [ ]:
rd= mli.get_range_dict(X_train)
mli.ice_plot(stb1, plot_pts, X_test.columns, rd, plots_per_row=2)